# 3D Poisson Equation

In [ ]:
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
import ngsolve

In [ ]:
geo = Box((0,0,0), (1,1,1))

geo.faces.name="outer"

DrawGeo(geo);

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.3)).Curve(3)
Draw (mesh);

In [ ]:
mesh.GetBoundaries()

## Poisson problem (Strong form)

Steady state heat: $u : \Omega \rightarrow R$

Linear strain:
$$
-\Delta u + u = f
$$

Boundary conditions:
$$
u = u_D \qquad \text{on} \, \Gamma_D\\
\frac{\partial u}{\partial n} = g \qquad \text{on} \, \Gamma_N
$$

Data:
$$
f = 1 \\
u_D = 0\\
g = 2
$$
Variational formulation:
--- 
Find: $u \in H^1(\Omega)$ such that $u = u_D$ on $\Gamma_D$
$$
\int_\Omega \nabla u . \nabla v \, dx + \int_\Omega u v \, dx = \int_\Omega f v dx + \int_{\Gamma_N} g v ds
$$
holds for all $v = 0$ on $\Gamma_D$.

In [ ]:
fes = H1(mesh, order=3)
u,v = fes.TnT()
gfu = GridFunction(fes)

with TaskManager():
    a = BilinearForm(grad(u)*grad(v)*dx+u*v*dx)
#     pre = Preconditioner(a, "bddc")
    a.Assemble()

In [ ]:
force = 1
g = x
f = LinearForm(force*v*dx + g*v*ds("outer")).Assemble()

In [ ]:
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

In [ ]:
# from ngsolve.krylovspace import CGSolver
# inv = CGSolver(a.mat, pre, printrates='\r', tol=1e-8)
# gfu.vec.data = inv * f.vec

In [ ]:
Draw (gfu, mesh);

In [ ]:
mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=0.0625)).Curve(3)
fes = H1(mesh, order=2)
u,v = fes.TnT()
exact_gfu = GridFunction(fes)

with TaskManager():
    a = BilinearForm(grad(u)*grad(v)*dx + u*v*dx)
    a.Assemble()
    force = 1
    g = x
    f = LinearForm(force*v*dx + g*v*ds("outer")).Assemble()
    exact_gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec

In [ ]:
Draw(exact_gfu)

In [ ]:
# from ngsolve.krylovspace import CGSolver

In [ ]:
mesh_size = [0.5, 0.25, 0.125,  0.0625, 0.03125] #, 0.015625]
dofs = []
errors = []

for h in mesh_size:
    mesh = Mesh(OCCGeometry(geo).GenerateMesh(maxh=h)).Curve(3)
    fes = H1(mesh, order=1)
    u,v = fes.TnT()
    gfu = GridFunction(fes)

    with TaskManager():
        a = BilinearForm(grad(u)*grad(v)*dx + u*v*dx)
#         pre = Preconditioner(a, "bddc")
        a.Assemble()
        force = 1
        g = x
        f = LinearForm(force*v*dx + g*v*ds("outer")).Assemble()
        gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
#         inv = CGSolver(a.mat, pre, printrates='\r', tol=1e-8)
#         gfu.vec.data = inv * f.vec
        dofs.append(fes.ndof)
        
    err = sqrt (Integrate ( (gfu-exact_gfu)*(gfu-exact_gfu), mesh))
    errors.append(err)


In [ ]:
errors

In [ ]:
import numpy as np
for i in range(len(mesh_size)-1):
    print("Convergence rate:", np.log(errors[i+1]/errors[i])/np.log(mesh_size[i+1]/mesh_size[i]))


In [ ]:
dofs = [21, 138, 920, 5786, 42316]

In [ ]:
adofs = [4.0, 16.0, 64.0, 256.0, 1024.0]
adofs = [1/h**2 for h in mesh_size]
print(adofs)

In [ ]:
errors = [0.03237782729474175,
 0.007454305253025955,
 0.0016647719003890133,
 0.0004531523960319344,
 0.00011365297264529204]

In [ ]:
import matplotlib.pyplot as plt
plt.loglog(dofs, np.divide(np.ones(len(dofs)), np.square(np.cbrt(np.array(dofs)))), label="Reference Line 1/dofs^(2/3)")
plt.loglog(dofs,errors, "-o", label="L2 Error")
plt.title("Error Estimation Plot")
plt.xlabel("Degrees of Freedom ")
plt.ylabel("L2 Error")
plt.legend()
plt.show()